### 文本转向量介绍


#### 如何使用bert模型将文本转换成向量的
1. 主要是使用hidden_status值、attention_mask 来做处理的。
2. 基本的参考链接可以看这里 https://github.com/UKPLab/sentence-transformers/blob/06f5c4e9857f013da2657d43a77d9f5f0bf50a61/sentence_transformers/models/Pooling.py#L128
3. 最常见的，或者最方便的，就是提取cls对应的hidden_status的值。当然，还有别的，这里先不展开介绍。


#### 标准化问题
1. 为什么有的时候是使用cos相似度，有的时候，就是直接用矩阵乘法
2. 实际上是等效的：
- 2.1 如果模型输出的时候，已经做过标准化了，那就直接使用矩阵乘法就行了。
- 2.2 如果模型输出的时候，没有做标准化，那就使用cos相似度

3. 做标准化部分，就相当于提前做了$A / ||A||$操作

![](images/cos.png)


## 如何使用文本转向量模型


In [1]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name_or_path = "model/bge-base-zh-v1.5"

In [2]:
# Sentences we want sentence embeddings for
sentences = ["样例数据-1", "样例数据-2"]

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModel.from_pretrained(model_name_or_path)
model.eval()

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
# for s2p(short query to long passage) retrieval task, add an instruction to query (not add instruction for passages)
# encoded_input = tokenizer([instruction + q for q in queries], padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)
    # Perform pooling. In this case, cls pooling.
    sentence_embeddings = model_output[0][:, 0]
# normalize embeddings
sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
print("Sentence embeddings:", sentence_embeddings)

Sentence embeddings: tensor([[-0.0157, -0.0291,  0.0919,  ..., -0.0066,  0.0219,  0.0269],
        [-0.0168, -0.0304,  0.1028,  ..., -0.0277,  0.0120,  0.0091]])


### onnx 推理转换

1. http://www.xavierdupre.fr/app/onnxcustom/helpsphinx/api/onnxruntime_python/helpers.html

```BASH
pip install "optimum[onnxruntime]" # cpu版本
pip install "optimum[onnxruntime-gpu]" # gpu版本

```

In [ ]:
import pprint
import onnxruntime

pprint.pprint(onnxruntime.get_available_providers())

In [4]:
model_output.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [9]:
model_output.last_hidden_state[:,0].shape

torch.Size([2, 768])